In [4]:
'''
Checks if guess is the same as answer
gives back an array, 
0 : wrong letter
1 : right letter, wrong spot
2 : right letter, right spot
'''
def checkWord(guess, ans):
    correct = [0, 0, 0, 0, 0]
    used = [' ', ' ', ' ', ' ', ' ']
    for i in range(len(ans)):
        if (guess[i] == ans[i]):
            correct[i] = 2
            used[i] = guess[i]
    
    for i in range(len(ans)):
        if (correct[i] == 2):
            continue
        if (ans.count(guess[i]) > 0 and used.count(guess[i]) == 0):
            correct[i] = 1
    
    return correct

def getWord():
    while(1):
        print("Enter a five letter word")
        guess = input()
        if (len(guess) == 5 and Root.IsValidWord(guess)):
            return guess
        print("Not a valid word")



In [16]:
def game(WordList: Trie):
    guesses = 0
    gameOver = False
    answer = WordList.GetRandomWord()
    while (guesses < 6):
        guess = getWord()
        print(guess)
        correct = checkWord(guess, answer)
        if (correct.count(2) == 5):
            gameOver = True
            break
        print(correct)
        guesses += 1

    if (gameOver):
        print("u win")
    if (not gameOver and guesses == 6):
        print("game over, u lost")

    print(answer)


In [17]:

from random import randint

class TrieNode:
    End : bool
    NextNodes : list

    def __init__(self):
        self.End = False
        self.NextNodes = [None] * 26
    
    def IsEnd(self):
        return self.End
    
    
    def GetNode(self, i : int):
        return self.NextNodes[i]

    def CreateNode(self, i : int):
        self.NextNodes[i] = TrieNode()
        return self.NextNodes[i]

from random import randint

class Trie:

    Root : TrieNode
    Count : 0

    def __init__(self):
        self.Root = TrieNode()
        self.Count = 0
    

    def Insert(self, word : str):
        self.Count += 1
        cur = self.Root
        for l in word:
            idx = ord(l) - 97 # 'a' in ASCII
            tmp = cur.GetNode(idx)
            if tmp is None:
                tmp = cur.CreateNode(idx)
            cur = tmp
        cur.End = True

    def Delete(self, word : str):
        self.Count -= 1
        cur = self.Root
        for l in word:
            cur = cur.GetNode(ord(l) - 97) # 'a' in ASCII
            if cur is None:
                raise TypeError("Word is not in Trie")
        cur.End = False

    def IsValidWord(self, word : str):
        cur = self.Root
        for l in word:
            cur = cur.GetNode(ord(l) - 97) # 'a' is ASCII
            if cur is None:
                return False
        return True

    # def GetAllWordsFitting(self, ):

    
    def GetRandomWord(self):
        len = 5
        cur = ""
        node = self.Root
        while len > 0:
            idx = randint(0, 25)
            tmp = node.GetNode(idx)
            if tmp is not None:
                len -= 1
                cur = cur + chr(idx + 97) # 'a' in ASCII
                node = tmp
        return cur

Root = Trie()

f = open("words.txt", "r")


for word in f:
    word = word[:-1]
    Root.Insert(word)


In [18]:

game(Root)

Enter a five letter word
thorn
[0, 0, 0, 1, 0]
Enter a five letter word
plane
[0, 0, 2, 0, 0]
Enter a five letter word
plane
[0, 0, 2, 0, 0]
Enter a five letter word
adieu
[1, 0, 0, 0, 0]
Enter a five letter word
qophs
[0, 0, 0, 0, 2]
Enter a five letter word
Not a valid word
Enter a five letter word
build
[0, 0, 0, 0, 0]
game over, u lost
xrays
